## Importar Librerías

In [1]:
from psycopg2 import connect, Error
import pandas as pd
import csv
from io import StringIO
import glob
import functools

## Conexión

In [2]:
def getConnection():
    try:
        connection = connect(host='localhost',database='amazon',user='postgres', password='1234', port='5432')   
    except(Exception, Error) as error:
        connection.rollback()
        print("Error: %s" % error)
    return connection

## Función para cargar tabla a la base de datos

In [3]:
def to_database(dataframe,table):
    try:
        output = StringIO()
        dataframe.to_csv(output, sep='\t', index = False, header = False, quoting=csv.QUOTE_NONE,escapechar='\\')
        output.seek(0)
        connection = getConnection()
        cursor = connection.cursor()
        cursor.copy_from(output,table,null='')
        connection.commit()
        cursor.close()
    except(Exception, Error) as error:
        print("Error: %s" % error)


## Crear DataFrame de los archivos

Se crea el DataFrame con los datos de todos los archivos terminados en ".tsv" del directorio

In [4]:
#pd.concat: Concatena todos los dataFrame creados, uno por cada archivo
#map: Iterador que aplica la funcion a todos los parametros entregados (Los nombres de los archivos)
#pd.read_csv: Lee el archivo .tsv        
#glob.glob: Encuentra el nombre de todos los archivos terminados en .tsv

df = pd.concat(map(functools.partial(pd.read_csv, sep='\t',on_bad_lines='skip', quoting = 3), glob.glob('*.tsv')))

## Tabla Categoria

Se crea un nuevo data frame con la columna "product_category", realizando el proceso de ETL para la posterior carga a la base de datos

In [15]:
df_category = df[["product_category"]]

cantTotal = df_category.count()
cantNull = df_category.isnull().sum()

df_category = df_category.drop_duplicates(subset="product_category")

cantRemplazados = df_category["product_category"].str.contains('_').sum()

df_category = df_category.replace('_',' ',regex = True)

df_category.dropna(how='any', inplace=True)

cantUnicos = df_category.count()

to_database(df_category,"category")

print("Cantidad total: ",int(cantTotal),"\nCantidad nulos: ",int(cantNull),"\nCantidad únicos: ",int(cantUnicos),"\nCantidad reemplazados: ",int(cantRemplazados))

df_category

Cantidad total:  34765547 
Cantidad nulos:  0 
Cantidad únicos:  23 
Cantidad reemplazados:  6


,product_category
0,Automotive
0,Baby
0,Camera
0,Digital Music Purchase
0,Digital Software
0,Digital Video Download
0,Digital Video Games
0,Furniture
0,Gift Card
0,Grocery


## Tabla Customer
Se crea un nuevo data frame con la columna "customer_id", realizando el proceso de ETL para la posterior carga a la base de datos

In [6]:
df_customers_id = df[["customer_id"]]

cantTotal = df_customers_id.count()
cantNull = df_customers_id.isnull().sum()

df_customers_id = df_customers_id.drop_duplicates(subset="customer_id")

cantUnicos = df_customers_id.count()

df_customers_id.dropna(how='any', inplace=True)

to_database(df_customers_id,"customer")

print("Cantidad total: ",int(cantTotal),"\nCantidad nulos: ",int(cantNull),"\nCantidad únicos: ",int(cantUnicos))

df_customers_id

Cantidad total:  34765547 
Cantidad nulos:  0 
Cantidad únicos:  12916044


,customer_id
0,36075342
1,42462164
2,21241933
3,52570308
4,38200102
...,...
960864,39578015
960867,39508379
960868,44704251
960869,40571775


## Tabla Time
Se crea un nuevo data frame con la columna "review_date", realizando el proceso de ETL para la posterior carga a la base de datos

In [10]:
df_time = df[["review_date"]]
cantTotal = df_time.count()
cantNull = df_time.isnull().sum()

df_time = df_time.drop_duplicates(subset="review_date")

cantUnicos = df_time.count()
df_time = df_time.dropna()

df_time[[ "year","month", "day"]] = df_time["review_date"].str.split("-", expand = True)

to_database(df_time,"time")

print("Cantidad total: ",int(cantTotal),"\nCantidad nulos: ",int(cantNull),"\nCantidad únicos: ",int(cantUnicos))
df_time

Cantidad total:  34765547 
Cantidad nulos:  0 
Cantidad únicos:  6506


,review_date,year,month,day
0,2015-08-31,2015,08,31
6507,2015-08-30,2015,08,30
11892,2015-08-29,2015,08,29
17486,2015-08-28,2015,08,28
21935,2015-08-27,2015,08,27
...,...,...,...,...
380599,1996-08-17,1996,08,17
380600,1996-06-11,1996,06,11
380601,1996-06-05,1996,06,05
380602,1995-12-29,1995,12,29


## Tabla Product
Se crea un nuevo data frame con la columnas "product_id", "product_parent", "product_title", "product_category" realizando el proceso de ETL para la posterior carga a la base de datos

In [18]:
df_products = df[["product_id", "product_parent", "product_title", "product_category"]]

cantTotal = df_products["product_id"].count()
cantNull = df_products["product_id"].isnull().sum()

df_products = df_products.drop_duplicates(subset="product_id")

cantRemplazados = df_products["product_category"].str.contains('_').sum()
cantUnicos = df_products["product_id"].count()

df_products["product_category"] = df_products["product_category"].replace('_',' ',regex = True)

csv_file = "product.csv"
table = "product"

df_products.to_csv(csv_file, sep = ';', index = False, header = False, quoting = csv.QUOTE_NONE, escapechar = '\\')

print("Cantidad total: ",int(cantTotal),"\nCantidad nulos: ",int(cantNull),"\nCantidad únicos: ",int(cantUnicos),"\nCantidad reemplazados: ",int(cantRemplazados))

df_products

Cantidad total:  34765547 
Cantidad nulos:  0 
Cantidad únicos:  5947719 
Cantidad reemplazados:  922596


,product_id,product_parent,product_title,product_category
0,B00LPRXQ4Y,339193102,"17"" 2003-2006 Ford EXPEDITION Ford F150 2004-2...",Automotive
1,B000C7S0TO,907684644,Spectra Premium CU1909 Complete Radiator for T...,Automotive
2,B000CO9WE4,752246352,K&N E-4665 High Performance Replacement Indust...,Automotive
3,B000GKD5NI,105401756,Suncutters Rear Window Shade,Automotive
4,B009SDA7TE,728471129,Lug Nuts Landcruiser Tundra OEM Mag 14x1.5 Thr...,Automotive
...,...,...,...,...
960819,B00008IM8V,967363878,Timex Dress Metals 23001,Watches
960829,B00006J6TZ,110036990,NaN,Watches
960834,B000066OSE,384427633,Timex Women's Fashion Watch 17961,Watches
960836,B00006441M,126100690,Fossil - FS2857 (Size: men),Watches


## Tabla Review
Se crea un nuevo data frame con los campos de review, realizando el proceso de ETL para la posterior carga a la base de datos

In [19]:
df_review = df[["marketplace", "customer_id", "review_id", "product_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase", "review_headline", "review_body","review_date"]]

cantTotal = df_review["review_id"].count()
cantNull = df_review["star_rating"].isnull().sum()

df_review.dropna(subset=["star_rating"], inplace = True)

csv_file = "review.csv"
table = "review"

df_review.to_csv(csv_file, sep = ';', index = False, header = False, quoting = csv.QUOTE_NONE, escapechar = '\\')

print("Cantidad total: ",int(cantTotal),"\nCantidad nulos: ",int(cantNull))

df_review

Cantidad total:  34765547 
Cantidad nulos:  0


C:\Users\valeb\AppData\Local\Temp\ipykernel_19804\242456140.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review.dropna(subset=["star_rating"], inplace = True)


,marketplace,customer_id,review_id,product_id,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,36075342,RAB23OVFNCXZQ,B00LPRXQ4Y,1,0,0,N,Y,"As it was used,","As it was used, the method that Ford used to a...",2015-08-31
1,US,42462164,R3NORADVJO6IE6,B000C7S0TO,5,0,0,N,Y,Five Stars,"Put it in fine, no problems. Shipping was dece...",2015-08-31
2,US,21241933,R299F4SO98S5OO,B000CO9WE4,5,1,1,N,Y,Great fit and performance on the surface drive...,Fit wonderfully on my 36HP Pro-Drive motor.,2015-08-31
3,US,52570308,R2DA9DOT03UW6I,B000GKD5NI,5,2,3,N,Y,Good for the price. Fits fairly good on 2010 T...,Good for the price! So far I have put it up i...,2015-08-31
4,US,38200102,R2OGCH681EQHU6,B009SDA7TE,5,0,0,N,Y,Five Stars,Fit perfectly on my 2012 Tundra with stock alu...,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...
960867,US,39508379,REH8UQZAXQS40,B00005OT9I,2,17,17,N,N,Great prize - but size??,It's a great watch - but you don't have a chan...,2001-12-13
960868,US,44704251,RJ14QPZEOI9P8,B00005QEM6,5,9,10,N,N,A very classy watch!,"Well worth the price. This is a light weight, ...",2001-11-08
960869,US,40571775,R2B1G5650WWFCE,B00005QEME,5,3,16,N,N,This is a great watch,"Dear Targeteers,<BR>This watch is exelent. it ...",2001-11-06
960870,US,44474855,R2MMGPUWXXOFI2,B00004YK0H,4,0,0,N,N,Now watt a minute here.,"In the old days, the common hearing battery in...",2001-04-05
